<a href="https://colab.research.google.com/github/chouhandiksha/bigdataproject/blob/main/notebooks/Analyse%20LA%20Compare%20Full%20Time%20Work%20Percentage%20For%20Wealthiest%20And%20Poorest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Spark SQL Documentation:** 
https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

# Analyse LA Compare Full Time Work Percentage For Wealthiest And Poorest

*Instructions:*

1. Execute the first code cell.
2. There will be a link to follow in order to authorize the google account for drive. Go to that link.
3. A code to authorize the google account will be generated. Copy the code generated.
4. Go back to the cell where the process of mounting the drive is running. Paste the generated code from step 3 to the text box in the cell and press enter.

In [57]:
# add time information at the end of every cell
!pip install ipython-autotime
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 5.85 s (started: 2021-05-03 17:44:30 +00:00)


In [58]:
# Install required dependancies
!pip install pyspark
!apt update
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [59]:
# Import modules
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from pathlib import Path
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

time: 12.9 ms (started: 2021-05-03 17:44:47 +00:00)


In [60]:
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

time: 7.71 ms (started: 2021-05-03 17:44:47 +00:00)


In [63]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

time: 216 ms (started: 2021-05-03 17:45:06 +00:00)


In [64]:
#sc.stop()

time: 1.36 ms (started: 2021-05-03 17:45:06 +00:00)


In [65]:
# Mount drive with data
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 10.7 ms (started: 2021-05-03 17:45:06 +00:00)


In [66]:
# Set path to data folder
path = Path('drive/MyDrive/big-data-project/data/clean-data')
city = 'la'

time: 2.13 ms (started: 2021-05-03 17:45:06 +00:00)


In [67]:
# Read data into dataframe
df_soc = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'social/2020/*.csv'))
df_soc.show()

+---+------------+--------------------+--------------------+------------+---------------------------+--------------------------+------------------------------------------+----------------------------+----------------------+------------------------+--------------------+-------------------------------+-------------------------------+--------------------+-------------------------+--------------------------+----------------------+----------------------------+---------------------------+-----------------------------+
|_c0|         cbg|    date_range_start|      date_range_end|device_count|distance_traveled_from_home|bucketed_distance_traveled|median_dwell_at_bucketed_distance_traveled|completely_home_device_count|median_home_dwell_time|bucketed_home_dwell_time|at_home_by_each_hour|part_time_work_behavior_devices|full_time_work_behavior_devices|    destination_cbgs|delivery_behavior_devices|median_non_home_dwell_time|candidate_device_count|bucketed_away_from_home_time|median_percentage_time_

In [68]:
df_soc.createOrReplaceTempView('clean_la')
df_soc = spark.sql('SELECT cbg, date_range_start as date, device_count, full_time_work_behavior_devices FROM clean_la WHERE device_count > 5')
df_soc.show()

+------------+--------------------+------------+-------------------------------+
|         cbg|                date|device_count|full_time_work_behavior_devices|
+------------+--------------------+------------+-------------------------------+
|060371831013|2020-02-07T00:00:...|          45|                              1|
|060374034022|2020-02-07T00:00:...|         155|                              4|
|060375326041|2020-02-07T00:00:...|          31|                              2|
|060375544031|2020-02-07T00:00:...|         130|                             10|
|060375718003|2020-02-07T00:00:...|          35|                              1|
|060590018024|2020-02-07T00:00:...|          38|                              4|
|060590320463|2020-02-07T00:00:...|         184|                             12|
|060590626281|2020-02-07T00:00:...|         108|                              1|
|060371374011|2020-02-07T00:00:...|          72|                              1|
|060372398012|2020-02-07T00:

In [69]:
df_soc = df_soc.withColumn("date",
    df_soc['date'].substr(0, 10))

time: 25.7 ms (started: 2021-05-03 17:45:13 +00:00)


In [70]:
df_soc.show()

+------------+----------+------------+-------------------------------+
|         cbg|      date|device_count|full_time_work_behavior_devices|
+------------+----------+------------+-------------------------------+
|060371831013|2020-02-07|          45|                              1|
|060374034022|2020-02-07|         155|                              4|
|060375326041|2020-02-07|          31|                              2|
|060375544031|2020-02-07|         130|                             10|
|060375718003|2020-02-07|          35|                              1|
|060590018024|2020-02-07|          38|                              4|
|060590320463|2020-02-07|         184|                             12|
|060590626281|2020-02-07|         108|                              1|
|060371374011|2020-02-07|          72|                              1|
|060372398012|2020-02-07|          41|                              4|
|060373009011|2020-02-07|          82|                              6|
|06037

In [71]:
# Add fulltime percentage column

df_soc = df_soc.withColumn('full_time_work_percentage', (df_soc['full_time_work_behavior_devices']/df_soc['device_count']) * 100)

#df_soc.show()

time: 48.6 ms (started: 2021-05-03 17:45:14 +00:00)


In [72]:
#Create temp view
df_soc.createOrReplaceTempView('mobility')

time: 19.6 ms (started: 2021-05-03 17:45:14 +00:00)




---


**Demographic data**





---



In [73]:
# Read poverty data
# Read data into RDD
df_demographic = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'la.csv'))
df_demographic.createOrReplaceTempView('demographic')
df_demographic.show()

+------------+---------+------------------+------------------+
|         cbg|pop_total|poverty_percentage|    perc_whiteonly|
+------------+---------+------------------+------------------+
|060373104003|     1286|17.884914463452567| 69.51788491446345|
|060590865023|     2545|22.789783889980352| 85.34381139489194|
|060376510024|     1285|               0.0|42.490272373540854|
|060376511011|     2708| 7.745398773006134| 50.70162481536189|
|060590994023|      672|12.969283276450511| 38.54166666666667|
|060590995094|      342| 17.83625730994152| 77.48538011695906|
|060376511012|     2483|2.6178010471204187|50.302053966975436|
|060376512011|      859|2.8846153846153846| 82.42142025611176|
|060376512012|     1179| 1.441899915182358|  74.1306191687871|
|060376512014|      989|2.4266936299292214| 70.57633973710819|
|060376512211|     3373|  9.95684340320592| 40.61666172546695|
|060590995092|      938| 5.756929637526652|  84.9680170575693|
|060590995095|      576|14.583333333333334|           7

In [74]:
df_mob_demo = spark.sql('SELECT mobility.*, demographic.poverty_percentage from mobility INNER JOIN demographic ON mobility.cbg = demographic.cbg')
df_mob_demo.createOrReplaceTempView('demographic_mobility')
df_mob_demo.show()


+------------+----------+------------+-------------------------------+-------------------------+------------------+
|         cbg|      date|device_count|full_time_work_behavior_devices|full_time_work_percentage|poverty_percentage|
+------------+----------+------------+-------------------------------+-------------------------+------------------+
|060371831013|2020-02-07|          45|                              1|       2.2222222222222223|15.450643776824036|
|060374034022|2020-02-07|         155|                              4|       2.5806451612903225| 10.67193675889328|
|060375326041|2020-02-07|          31|                              2|        6.451612903225806| 27.79187817258883|
|060375544031|2020-02-07|         130|                             10|       7.6923076923076925|  25.3447633246366|
|060375718003|2020-02-07|          35|                              1|        2.857142857142857|3.1537450722733245|
|060590018024|2020-02-07|          38|                              4|  

In [75]:
# DIVIDING THE PEOPLE TOP 20 PERCENT AND BOTTOM 20 PERCENT

df_mob_demo_high_class =  spark.sql('SELECT * FROM demographic_mobility WHERE poverty_percentage < 20')
grouped_df_mob_demo_high_class = df_mob_demo_high_class.groupBy("date").mean("full_time_work_percentage").withColumnRenamed('avg(full_time_work_percentage)','full_time_work_percentage')
grouped_df_mob_demo_high_class.createOrReplaceTempView('mob_demo_high_class')


df_mob_demo_low_class =  spark.sql('SELECT * FROM demographic_mobility WHERE poverty_percentage > 80')
grouped_df_mob_demo_low_class = df_mob_demo_low_class.groupBy("date").mean("full_time_work_percentage").withColumnRenamed('avg(full_time_work_percentage)','full_time_work_percentage')
grouped_df_mob_demo_low_class.createOrReplaceTempView('mob_demo_low_class')

grouped_df_mob_demo_combined = spark.sql("""

SELECT h.date,h.full_time_work_percentage as top20_perc,
l.full_time_work_percentage as bottom20_perc  
FROM mob_demo_high_class h INNER JOIN mob_demo_low_class l ON h.date = l.date

""")


time: 159 ms (started: 2021-05-03 17:45:15 +00:00)


**Load 2019 stats**

In [76]:
filename = '{}_social_summary_2019.csv'.format(city)
stat_2019 = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/filename))
stat_2019.show()
mean_2019 = stat_2019.collect()[1][7]


+-----+------------------+-----------------+-----------------+-----------------+--------------------------+-------------------------+-------------------------+--------------------+--------------------+--------------------+------------------+
|  _c0|             total|  completely_home|   part_time_work|   full_time_work|percentage_completely_home|percentage_part_time_work|percentage_full_time_work|norm_completely_home| norm_part_time_work| norm_full_time_work|             month|
+-----+------------------+-----------------+-----------------+-----------------+--------------------------+-------------------------+-------------------------+--------------------+--------------------+--------------------+------------------+
|count|         3004769.0|        3004769.0|        3004769.0|        3004769.0|                 3004769.0|                3004769.0|                3004769.0|           3004769.0|           3004769.0|           3004769.0|         3004769.0|
| mean| 87.84796368705881|27.763

In [77]:
# calculate deviation form mean
grouped_df_mob_demo_combined= grouped_df_mob_demo_combined.withColumn("top20_deviation",
    grouped_df_mob_demo_combined['top20_perc'] - mean_2019)


time: 21.9 ms (started: 2021-05-03 17:45:15 +00:00)


In [78]:
grouped_df_mob_demo_combined = grouped_df_mob_demo_combined.withColumn("bottom20_deviation",
    grouped_df_mob_demo_combined['bottom20_perc'] - mean_2019)

time: 19.4 ms (started: 2021-05-03 17:45:15 +00:00)



**Visualizations** 



In [79]:
# create altair library theme 

def my_theme():
  return {
    'config': {
      'view': {'continuousHeight': 300, 'continuousWidth': 400},  # from the default theme
      'range': {'category': {'scheme': ['#FF4500','#a0aab4','#4E79A7']}}
    }
  }
alt.themes.register('my_theme', my_theme)
alt.themes.enable('my_theme')
# 

ThemeRegistry.enable('my_theme')

time: 13.5 ms (started: 2021-05-03 17:45:15 +00:00)


In [80]:
def create_chart(df,mon) :
  chart = alt.Chart(df).transform_fold(
    fold=['Bottom 20 % Deviation', 'Top 20 % Deviation'],
    as_ = ['Poverty %','full_time_value']
    
    ).mark_line().encode(
      x= alt.X('date:T', title = mon),
      y= alt.Y('full_time_value:Q', title = 'Completely Home %'),
      color='Poverty %:N'
    ).properties(width = 1000)

  rule = alt.Chart(pd.DataFrame({'Mean 2019':[0]})).mark_rule(color='#757575',strokeDash=[5,3], size=2).encode(
    y='Mean 2019',
    size=alt.value(2),
    )
  
  text = alt.Chart({'values':[{ 'y': 1}]}).mark_text(
    text='Mean 2019', angle=0
    ).encode(
      y=alt.Y('y:Q'),opacity=alt.value(0.4)
    )

  return  chart + rule + text


time: 22.6 ms (started: 2021-05-03 17:45:15 +00:00)


In [81]:
def apply_date_filter(df,start_date,end_date) :
  mask = (df['date'] > start_date) & (df['date'] <= end_date)
  dfX = df.loc[mask]
  return dfX

time: 6.97 ms (started: 2021-05-03 17:45:15 +00:00)


In [82]:
# create pandas dataframe for visualizations
df = grouped_df_mob_demo_combined.toPandas()
df

,date,top20_perc,bottom20_perc,top20_deviation,bottom20_deviation
0,2020-02-26,7.436867,2.038400,2.162698,-3.235769
1,2020-04-13,4.278560,2.352525,-0.995609,-2.921644
2,2020-06-24,4.409102,3.739318,-0.865067,-1.534851
3,2020-06-08,4.794649,3.525617,-0.479520,-1.748552
4,2020-09-12,3.810222,3.833597,-1.463947,-1.440572
...,...,...,...,...,...
361,2020-04-05,2.545564,1.740760,-2.728605,-3.533408
362,2020-05-01,3.770745,2.707222,-1.503423,-2.566946
363,2020-10-25,3.662916,7.176751,-1.611252,1.902582
364,2020-12-28,5.613756,4.064940,0.339587,-1.209228


time: 3min 14s (started: 2021-05-03 17:45:15 +00:00)


In [83]:
df = df.rename(columns={'top20_perc': 'Top 20 %', 'bottom20_perc': 'Bottom 20 %','top20_deviation': 'Top 20 % Deviation', 'bottom20_deviation': 'Bottom 20 % Deviation'})

time: 20.7 ms (started: 2021-05-03 17:48:30 +00:00)


In [84]:
# analysis for the entire year on monthly basis
df_str = df.copy(deep=True) #make a deep copy

df_str['date'] = pd.to_datetime(df_str['date'],utc= True) # extract the months

df_str = df_str.groupby([df_str['date']]).mean().rolling(10).mean()[0:] #find the mean for the days and roll up for every 10 days

chart = alt.Chart(df_str.reset_index()).transform_fold(
    fold=['Bottom 20 %', 'Top 20 %'],
    as_ = ['Poverty %','full_time_value']
    
    ).mark_line().encode(
      x='date:T',
      y= alt.Y('full_time_value:Q', title = 'Full Time %'), color='Poverty %:N'
    ).properties(width = 1000)
chart

alt.Chart(...)

time: 98.3 ms (started: 2021-05-03 17:48:30 +00:00)


In [85]:
#Conversion of datetime format
df['date'] = pd.to_datetime(df['date'],utc= True)


time: 9.44 ms (started: 2021-05-03 17:48:30 +00:00)


Filtering based on important dates



*   Thursday, March 19, 2020: Statewide Stay-at-Home Order Issued











In [86]:
start_date = '2020-3-16'
end_date = '2020-4-9'
df1 = apply_date_filter(df,start_date,end_date)
chart = create_chart(df1,'March-April')
chart

alt.LayerChart(...)

time: 54.2 ms (started: 2021-05-03 17:48:30 +00:00)





*   August. 7, 2020: LA County Exceeds 200,000 Coronavirus Cases
*   August, 12, 2020: LA County Crosses 'Tragic Milestone' Of 5,000 Coronavirus Deaths 









In [87]:
start_date = '2020-8-5'
end_date = '2020-8-19'
df2 = apply_date_filter(df,start_date,end_date)
chart = create_chart(df2,'August')
chart
#df2

alt.LayerChart(...)

time: 64 ms (started: 2021-05-03 17:48:30 +00:00)


*   Oct. 14, 2020: LA County Sees Rise In Workplace Outbreaks As Infection Rate Creeps Upward





In [88]:
start_date = '2020-10-12'
end_date = '2020-10-24'
df3 = apply_date_filter(df,start_date,end_date)
chart = create_chart(df3,'October')
chart

alt.LayerChart(...)

time: 65.9 ms (started: 2021-05-03 17:48:30 +00:00)


Dec. 16, 2020:
LA County COVID-19 Deaths Hit New Record


In [89]:
start_date = '2020-12-15'
end_date = '2020-12-26'
df4 = apply_date_filter(df,start_date,end_date)
chart = create_chart(df4,'December')
chart

alt.LayerChart(...)

time: 62.6 ms (started: 2021-05-03 17:48:30 +00:00)
